### GCN：从图数据中提取特征的方法 -- 带分类操作
- 从而可以使用这些特征去对图数据进行
    - 节点分类（node classification）
    - 图分类（graph classification）
    - 边预测（link prediction）
    - 图的嵌入表示（graph embedding）
- 结合应用点特征、点连接关系、点属性

In [2]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np
import scipy.sparse as sp
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

### 数据预处理

In [3]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [4]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))  #  矩阵行求和
    r_inv = np.power(rowsum, -1).flatten()  # 求和的-1次方
    r_inv[np.isinf(r_inv)] = 0.   # 如果是inf，转换成0
    r_mat_inv = sp.diags(r_inv)  # 构造对角戏矩阵
    mx = r_mat_inv.dot(mx)  # 构造D-1*A，非对称方式，简化方式
    return mx

In [5]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

### Cora Dataset
    该数据集共2708个样本点，每个样本点都是一篇科学论文，所有样本点被分为7个类别，类别分别是：
        1）基于案例；2）遗传算法；3）神经网络；4）概率方法；5）强化学习；6）规则学习；7）理论

    每篇论文都由一个1433维的词向量表示，即有1433个特征。词向量的每个元素都对应一个词，且该元素只有0或1两个取值。

    每篇论文都至少引用了一篇其他论文，或者被其他论文引用，也就是样本点之间存在联系，如果将样本点看做图中的点，则这是一个连通的图。
       
    cora.content：每一行由三部分组成，分别是论文的编号 | 论文的词向量 | 论文的类别
    
    cora.cites：每一行有两个论文编号，第一个是被引用论文，第二个是论文引用前一篇论文

In [6]:
def load_data(path=r"C:\Users\sss\Desktop\cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)  # 取特征feature
    labels = encode_onehot(idx_features_labels[:, -1])  # one-hot label

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)  # 节点
    idx_map = {j: i for i, j in enumerate(idx)}   # 构建节点的索引字典
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),  # 导入edge的数据
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)    # 将之前的转换成字典编号后的边
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),  # 构建边的邻接矩阵
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix，计算转置矩阵。将有向图转成无向图
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)   # 对特征做了归一化的操作
    adj = normalize(adj + sp.eye(adj.shape[0]))   # 对A+I归一化
    
    # 训练，验证，测试的样本
    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)
    # 将numpy的数据转换成torch格式
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [7]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [8]:
print("# 文章的矩阵关系 \t # 特征 \t # 标签")
print(adj.shape, "\t", features.shape, "\t", labels.shape)

# 文章的矩阵关系 	 # 特征 	 # 标签
torch.Size([2708, 2708]) 	 torch.Size([2708, 1433]) 	 torch.Size([2708])


### 模型准备

In [9]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))  # input_features, out_features
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    # 标准化
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)  # 随机化参数
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)  # GraphConvolution forward。input*weight
        output = torch.spmm(adj, support)  # 稀疏矩阵的相乘，和mm一样的效果
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [10]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(nfeat, nhid) # 构建第一层 GCN
        self.gc2 = GraphConvolution(nhid, nclass) # 构建第二层 GCN
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [11]:
hidden = 16
dropout = 0.5

model = GCN(
    nfeat=features.shape[1],
    nhid=hidden,
    nclass=labels.max().item() + 1,
    dropout=dropout
)

In [12]:
model

GCN(
  (gc1): GraphConvolution (1433 -> 16)
  (gc2): GraphConvolution (16 -> 7)
)

### 模型参数

In [11]:
seed = 42
cuda = not False and torch.cuda.is_available()
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [12]:
if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

### 训练和测试

In [13]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [14]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad() # GraphConvolution forward
    # 重点输出 ！！！
    output = model(features, adj)   # 运行模型，输入参数 (features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not False:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [15]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [16]:
t_total = time.time()
for epoch in range(200):
    train(epoch)
print("Optimization Finished")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss_train: 1.9502 acc_train: 0.1500 loss_val: 1.9400 acc_val: 0.1267 time: 9.7513s
Epoch: 0002 loss_train: 1.9274 acc_train: 0.1500 loss_val: 1.9243 acc_val: 0.1267 time: 0.0760s
Epoch: 0003 loss_train: 1.9181 acc_train: 0.1500 loss_val: 1.9098 acc_val: 0.1267 time: 0.0389s
Epoch: 0004 loss_train: 1.9130 acc_train: 0.1643 loss_val: 1.8963 acc_val: 0.1267 time: 0.0429s
Epoch: 0005 loss_train: 1.8930 acc_train: 0.1786 loss_val: 1.8834 acc_val: 0.1267 time: 0.2114s
Epoch: 0006 loss_train: 1.8848 acc_train: 0.1929 loss_val: 1.8712 acc_val: 0.1900 time: 0.0549s
Epoch: 0007 loss_train: 1.8662 acc_train: 0.2786 loss_val: 1.8595 acc_val: 0.4400 time: 0.0443s
Epoch: 0008 loss_train: 1.8598 acc_train: 0.3143 loss_val: 1.8483 acc_val: 0.3533 time: 0.0472s
Epoch: 0009 loss_train: 1.8493 acc_train: 0.3071 loss_val: 1.8375 acc_val: 0.3467 time: 0.0310s
Epoch: 0010 loss_train: 1.8275 acc_train: 0.3214 loss_val: 1.8268 acc_val: 0.3500 time: 0.0362s
Epoch: 0011 loss_train: 1.8331 acc_train

Epoch: 0087 loss_train: 0.8869 acc_train: 0.8000 loss_val: 1.0562 acc_val: 0.7733 time: 0.0278s
Epoch: 0088 loss_train: 0.9004 acc_train: 0.8286 loss_val: 1.0468 acc_val: 0.7767 time: 0.0423s
Epoch: 0089 loss_train: 0.8835 acc_train: 0.7786 loss_val: 1.0375 acc_val: 0.7767 time: 0.0395s
Epoch: 0090 loss_train: 0.8904 acc_train: 0.7714 loss_val: 1.0285 acc_val: 0.7767 time: 0.0264s
Epoch: 0091 loss_train: 0.8940 acc_train: 0.8000 loss_val: 1.0201 acc_val: 0.7800 time: 0.0258s
Epoch: 0092 loss_train: 0.8185 acc_train: 0.8500 loss_val: 1.0123 acc_val: 0.7833 time: 0.0259s
Epoch: 0093 loss_train: 0.8215 acc_train: 0.8714 loss_val: 1.0041 acc_val: 0.7833 time: 0.0507s
Epoch: 0094 loss_train: 0.8107 acc_train: 0.8429 loss_val: 0.9962 acc_val: 0.7833 time: 0.0388s
Epoch: 0095 loss_train: 0.7727 acc_train: 0.8357 loss_val: 0.9885 acc_val: 0.7833 time: 0.0302s
Epoch: 0096 loss_train: 0.7780 acc_train: 0.8500 loss_val: 0.9808 acc_val: 0.7833 time: 0.0300s
Epoch: 0097 loss_train: 0.7522 acc_train

Epoch: 0177 loss_train: 0.3960 acc_train: 0.9143 loss_val: 0.7083 acc_val: 0.8000 time: 0.0335s
Epoch: 0178 loss_train: 0.3927 acc_train: 0.9429 loss_val: 0.7078 acc_val: 0.8000 time: 0.0384s
Epoch: 0179 loss_train: 0.3983 acc_train: 0.9500 loss_val: 0.7070 acc_val: 0.8000 time: 0.0419s
Epoch: 0180 loss_train: 0.4297 acc_train: 0.9143 loss_val: 0.7063 acc_val: 0.8067 time: 0.0341s
Epoch: 0181 loss_train: 0.3729 acc_train: 0.9500 loss_val: 0.7052 acc_val: 0.8100 time: 0.0259s
Epoch: 0182 loss_train: 0.4353 acc_train: 0.9214 loss_val: 0.7025 acc_val: 0.8100 time: 0.0279s
Epoch: 0183 loss_train: 0.4385 acc_train: 0.9429 loss_val: 0.6983 acc_val: 0.8133 time: 0.0423s
Epoch: 0184 loss_train: 0.3867 acc_train: 0.9571 loss_val: 0.6954 acc_val: 0.8167 time: 0.0359s
Epoch: 0185 loss_train: 0.3816 acc_train: 0.9643 loss_val: 0.6941 acc_val: 0.8167 time: 0.0480s
Epoch: 0186 loss_train: 0.3960 acc_train: 0.9571 loss_val: 0.6938 acc_val: 0.8167 time: 0.0338s
Epoch: 0187 loss_train: 0.3559 acc_train

In [17]:
test()

Test set results: loss= 0.7037 accuracy= 0.8370
